# 1 задание

В наличии данные по карточным транзакциям клиентов за 3 месяца. 

Задача:
Выделить клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев.

Упрощенный пример поиска транзакций по заданной теме на примере транзакций в аптеках:
Мы ищем в столбце merchant_name наименования с ключевым словом “аптека” и известные сети (Ригла, Живика, Самсон-Фарма и др.), дополнительно ограничивая результат мсс, соответствующим аптекам (5912 и 5122). В результате получаем набор транзакций в аптеках.

Описание полей в файле schema.txt.

In [1]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql import functions as f

spark = SparkContext.getOrCreate(SparkConf())
sqlContext = SQLContext(spark)

In [2]:
df = sqlContext.read.orc("transactions_big.orc")

Для удобного вывода объявляю функцию _show:

In [3]:
import pandas as pd

def _show(df,cnt=10):
    return pd.DataFrame(df.take(cnt),columns=df.columns)

In [4]:
_show(df)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,0118DB6FC27F744F702C2B8C3B3C7C80,3A31094D3431D7CBC4B2B9560BA4C500,2019-10-30,264.00,810,5814,kofeynya novosibirsk ru,novosibirsk
1,0870E6D8C351C1EF4E67E2B6B72D04D5,24664CBAF7ABA3FD41BB19FC28BE5623,2019-09-25,135.00,810,5499,parle market moscow ru,moscow
2,0870E6D8C351C1EF4E67E2B6B72D04D5,24664CBAF7ABA3FD41BB19FC28BE5623,2019-09-05,420.00,810,5812,mercury cafe moscow ru,moscow
3,0870E6D8C351C1EF4E67E2B6B72D04D5,24664CBAF7ABA3FD41BB19FC28BE5623,2019-08-15,760.00,810,5814,farsh burgernaya moscow ru,moscow
4,0870E6D8C351C1EF4E67E2B6B72D04D5,24664CBAF7ABA3FD41BB19FC28BE5623,2019-08-16,596.42,810,5411,vkusvill 1343_4 moscow ru,moscow
5,17EDB6BEE33B1BAAC1FEFE79131A6D4B,205E1C63E7811F746CACAC7AC5B19B1A,2019-10-26,122.00,810,5814,ip varfolomeev i.n. omsk ru,omsk
6,18BE27D4C9C79140E4E44EE6FC309820,F85DC841CEC47D952C10B4C342EAF2B7,2019-08-13,65.00,810,5499,1000 melochej aldan ru,aldan
7,19FBB38873E61DC771DFCD65E3CEF098,A270F712B53132758ADCE7FD0F05429F,2019-09-24,195.00,810,5411,gurman artem ru,artem
8,19FBB38873E61DC771DFCD65E3CEF098,A270F712B53132758ADCE7FD0F05429F,2019-10-30,227.10,810,5411,evgenia artem ru,artem
9,1C99252D31CD143176DBB168EA19CCA7,43879A0FD09EA15FA630D77D3C425AF7,2019-10-18,749.00,810,5816,google*wg group ltd internet us,internet


In [5]:
df.dtypes

[('customer_id', 'string'),
 ('card_id', 'string'),
 ('tr_datetime', 'timestamp'),
 ('tr_normalized_amount', 'double'),
 ('currency', 'int'),
 ('mcc', 'int'),
 ('merchant_name', 'string'),
 ('merchant_city', 'string')]

Изменить столбец merchant_name в нижний регистр и убрать лишние пробелы

In [6]:
df = df.withColumn("merchant_name", f.trim(f.lower("merchant_name")))

Выделим номер mcc, у которых в merchant_name есть слово "cofee":

In [7]:
df_merchant_cofee = df.select("merchant_name",f.col("mcc")).where("merchant_name like '%cofee%'")

In [8]:
res_1 = df_merchant_cofee.groupby("merchant_name","mcc").count().orderBy(f.col("count").desc())

In [9]:
_show(res_1)

,merchant_name,mcc,count
0,cofee point moskva - 45-ru,5814,25
1,cofeemania terminal e moskva regioru,5814,15
2,travelers cofee krasnoyarsk ru,5814,10
3,i cofee moscow ru,5812,9
4,cofee point moskva ru,5814,7
5,public cofee tambov ru,5814,3
6,khleb and cofee moskovsk.oblru,5814,2
7,baggins cofee sankt-peterbru,5814,2
8,cofee plus novosibirsk ru,5814,2
9,bakery&cofee zern.7 st petersburru,5814,1


Выделяем mcc (5814, 5812)

In [11]:
df_merchant_cofee_2 = df.select(f.trim(f.lower("merchant_name")).alias("merchant_name"),f.col("mcc")).where("mcc in (5814, 5812)")
res_2 = df_merchant_cofee_2.groupby("merchant_name","mcc").count().orderBy(f.col("count").desc())

In [12]:
_show(res_2, 20)

,merchant_name,mcc,count
0,eurest stolovaya moskva ru,5814,4090
1,ors nlmk g lipetsk ru,5814,3795
2,domodedovo catering se domodedovo ru,5814,3685
3,restoran vsevolozhsk ru,5814,3620
4,domodedovo catering domodedovo ru,5814,3325
5,vendex sankt-peterbru,5814,1811
6,urest canteen moskva ru,5814,1787
7,compass group rus rosb moskva ru,5814,1710
8,pravda kofe moscow ru,5812,1220
9,russkiy appetit lipeczk ru,5814,1185


Дополнительно выделяем слова cofix, coffeeport, kofe, coffee с учётом mcc = (5814, 5812):

In [13]:
df_coffee = df.where("((merchant_name like '%cofee%') or (merchant_name like '%cofix%') or (merchant_name like '%coffeeport%') or (merchant_name like '%kofe%') or (merchant_name like '%coffee%')) and mcc in (5814, 5812)")

Определим по какому промежутку дат имеем транзакции:

In [14]:
df_coffee.select(f.min(f.col("tr_datetime")), f.max(f.col("tr_datetime"))).show()

+-------------------+-------------------+
|   min(tr_datetime)|   max(tr_datetime)|
+-------------------+-------------------+
|2019-08-01 00:00:00|2019-10-31 00:00:00|
+-------------------+-------------------+



Выделим в дате номер месяца транзакции:

In [15]:
df_coffee = df_coffee.withColumn("month", f.month(f.col("tr_datetime")))

In [16]:
_show(df_coffee)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city,month
0,0118DB6FC27F744F702C2B8C3B3C7C80,3A31094D3431D7CBC4B2B9560BA4C500,2019-10-30,264.0,810,5814,kofeynya novosibirsk ru,novosibirsk,10
1,41C181FED7463B4670BD0563F564111D,FF2AD226D11BB6ABC4695030436190F6,2019-10-02,100.0,810,5814,cofix hayrov 1 moscow ru,moscow,10
2,B2762F523B2B97002D38AC8ACC54D3EC,10AACE5B230E832740C449919F4431AC,2019-08-27,498.0,810,5814,kofeynya melnitsa yuzhno-sakharu,yuzhno-sakhalinsk,8
3,847C43F9AEF129B31946956CD63F425B,E41494CD110F53941433C5CF7413BE18,2019-09-06,218.0,810,5814,donats & kofe kol krasnodar ru,krasnodar,9
4,5BF9DED2A648E313B83A39C99ED18B99,160CD2E22E74845E996B708AB4A689B5,2019-09-17,145.0,810,5814,coffee point moskva ru,moscow,9
5,EDE930B92969E3BE0DFA951C03EB5CAE,88B7FE49EB8FF0471E12DF0F52E3273D,2019-10-10,2100.0,810,5812,coffeemania kutuzov moscow ru,moscow,10
6,3422307EDB2B170C378191368389CA88,EA6A33608F0753BE97D3AE06A9985505,2019-09-21,340.0,810,5814,kofe s soboj moscow ru,moscow,9
7,0405815580CA4D1A190D4E04DA422485,6A8A272030EFA2810FFDB7CF59E34966,2019-08-01,110.0,810,5814,kofe 1 domodedovo ru,domodedovo,8
8,A9BEB2BF5BE0E33F4522F3B36863990F,1FB794F222B19F9E3FBBD5AA2F09D59E,2019-10-05,420.0,810,5814,coffee house 55 moskow ru,moscow,10
9,725008C6C052938438A0C9A141CFACE8,7609F5B0560F8D673CEEA3D3C3316C7D,2019-09-13,200.0,810,5814,coffeeport moskva ru,moscow,9


Количество покупок кофе за месяц:

In [17]:
df_coffe_id_month = df_coffee.groupby("customer_id","month").count()
df_coffe_id_month.show(10)

+--------------------+-----+-----+
|         customer_id|month|count|
+--------------------+-----+-----+
|5A1B1797218B16661...|   10|    3|
|48960D01ED58B2483...|    8|    1|
|77793820997B64B13...|    9|    1|
|A7BE846C68F7A9CEF...|   10|    6|
|9C75A3E0407FEDEE5...|   10|    3|
|6A40E84331D016032...|   10|    9|
|9D541501834E3AA56...|   10|    3|
|67E0AFB345AB1925F...|    9|    2|
|EC478A08AEDC0E63F...|   10|    7|
|F120D047B50AC89C6...|    8|    5|
+--------------------+-----+-----+
only showing top 10 rows



Количество месяцев у клиента, в которых 2 и более покупки кофе:

In [18]:
df_coffe_id_month_2 = df_coffe_id_month.where("count > 1").groupby("customer_id").count()
df_coffe_id_month_2.show(10)

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|84B5E4F2D15E31E83...|    2|
|0AFB3E1D680AC62DB...|    1|
|09B403518CB24EBB3...|    1|
|66537676D709F8550...|    1|
|7D3531660DCE9D79D...|    2|
|2A1EE63DB31CBDFB5...|    1|
|AE226772B5D0FDAD2...|    2|
|E4C5D0B32AB8C2AFF...|    1|
|BCB6ED1BC854A431F...|    3|
|2597336A04BDD3CCC...|    1|
+--------------------+-----+
only showing top 10 rows



Выделяем id клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев:

In [19]:
df_customer_id = df_coffe_id_month_2.where("count > 1").select("customer_id").distinct().show(10)

+--------------------+
|         customer_id|
+--------------------+
|84B5E4F2D15E31E83...|
|7D3531660DCE9D79D...|
|AE226772B5D0FDAD2...|
|BCB6ED1BC854A431F...|
|93A75144A2F710322...|
|473FE84536E6BBE82...|
|044B6434B8A900DE9...|
|9DD2DAC4484A696D5...|
|E4E55A52BEA1AE4C4...|
|B3DB448056A25A341...|
+--------------------+
only showing top 10 rows



df_customer_id - id клиентов, которые покупали кофе/посещали кофейни от 2 раз в месяц хотя бы 2 из 3 месяцев.